In [1]:
!pip install mediapipe opencv-python

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [ ]:
#video feed
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    cv2.imshow('Mediapipe Feed',frame)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
        
        # Print results (you can access landmarks via results.pose_landmarks)
        print(results)

        # Display the image (can also draw landmarks if needed)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



In [ ]:
results.pose_landmarks

In [ ]:
mp_pose.POSE_CONNECTIONS

In [ ]:
mp_drawing.draw_landmarks??

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px">


In [ ]:
import cv2
import mediapipe as mp

mp_pose = mp.solutions.pose

cap = cv2.VideoCapture(0)

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break

        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        # Make detection
        results = pose.process(image)

        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #extract landmarks
        try:
            landmarks=results.pose_landmarks.landmark
            print(landmarks)
        except:
            pass

        #render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                 mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                                 )
        
        # Print results (you can access landmarks via results.pose_landmarks)
        print(results)

        # Display the image (can also draw landmarks if needed)
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()



In [ ]:
if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark
    print("Number of landmarks:", len(landmarks))


In [ ]:
for ldmrk in mp_pose.PoseLandmark:
    print(ldmrk)

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]

In [ ]:
landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

In [ ]:
def calculate_angle(a,b,c):
    a=np.array(a)#first
    b=np.array(b)#mid
    c=np.array(c)#end

    radians=np.arctan2(c[1]-b[1],c[0]-b[0])-np.arctan2(a[1]-b[1],a[0]-b[0])
    angle=np.abs(radians*180.0/np.pi)

    if angle>180.0:
        angle=360-angle

    return angle

In [ ]:
shoulder=[landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
elbow=[landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
wrist=[landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

In [ ]:
shoulder,elbow,wrist

In [ ]:
calculate_angle(shoulder,elbow,wrist)

In [ ]:
np.multiply(elbow,[640,480]).astype(int)

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point (elbow in this case)
    """
    a = np.array(a)  # First point (shoulder)
    b = np.array(b)  # Vertex point (elbow)
    c = np.array(c)  # End point (wrist)
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Initialize video capture
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates (normalized)
                shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                
                # Calculate angle
                angle = calculate_angle(shoulder, elbow, wrist)
                
                # Convert elbow coordinates to pixel coordinates
                elbow_coords = tuple(np.multiply(elbow, [frame_width, frame_height]).astype(int))
                
                # Draw circle and put angle text (using 'deg' instead of degree symbol)
                cv2.circle(image, elbow_coords, 10, (0, 255, 0), -1)
                cv2.putText(image, f'{int(angle)} deg',
                           (elbow_coords[0] - 40, elbow_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Curl counter logic (properly indented within the try block)
                if angle > 160:
                    stage = 'down'
                if angle < 30 and stage == 'down':
                    stage = "up"
                    counter += 1
                    print(f"Curl count: {counter}")
                
        except Exception as e:
            print("Error:", e)
        
        # Setup status box for displaying counter and stage
        cv2.rectangle(image, (0, 0), (300, 100), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), (15, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (150, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(stage), (150, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Display the image
        cv2.imshow('Bicep Curl Counter', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

Curl count: 1
Curl count: 2
Curl count: 3
Curl count: 4
Curl count: 5
Curl count: 6
Curl count: 7
Curl count: 8
Curl count: 9


KeyboardInterrupt: 

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point (knee in this case)
    """
    a = np.array(a)  # First point (hip)
    b = np.array(b)  # Vertex point (knee)
    c = np.array(c)  # End point (ankle)
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Initialize video capture
cap = cv2.VideoCapture(0)

# Squat counter variables
counter = 0
stage = None

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates for left leg (normalized)
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                # Get coordinates for right leg (normalized)
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Calculate angles for both knees
                left_angle = calculate_angle(left_hip, left_knee, left_ankle)
                right_angle = calculate_angle(right_hip, right_knee, right_ankle)
                
                # Use average of both knee angles for squat detection
                avg_angle = (left_angle + right_angle) / 2
                
                # Convert knee coordinates to pixel coordinates for display
                left_knee_coords = tuple(np.multiply(left_knee, [frame_width, frame_height]).astype(int))
                right_knee_coords = tuple(np.multiply(right_knee, [frame_width, frame_height]).astype(int))
                
                # Draw circles and put angle text at both knees
                cv2.circle(image, left_knee_coords, 8, (0, 255, 0), -1)
                cv2.putText(image, f'{int(left_angle)} deg',
                           (left_knee_coords[0] - 50, left_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.circle(image, right_knee_coords, 8, (0, 255, 0), -1)
                cv2.putText(image, f'{int(right_angle)} deg',
                           (right_knee_coords[0] + 10, right_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Squat counter logic
                # When standing up (knees extended)
                if avg_angle > 160:
                    stage = 'up'
                
                # When squatting down (knees bent) and previous stage was up
                if avg_angle < 90 and stage == 'up':
                    stage = 'down'
                    counter += 1
                    print(f"Squat count: {counter}")
                
        except Exception as e:
            print("Error:", e)
        
        # Setup status box for displaying counter and stage
        cv2.rectangle(image, (0, 0), (350, 120), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'SQUATS', (15, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(counter), (15, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STAGE', (150, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        cv2.putText(image, str(stage), (150, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Average angle data
        try:
            cv2.putText(image, 'AVG ANGLE', (240, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'{int(avg_angle)} deg', (240, 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        except:
            pass
        
        # Render pose detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Add instructions text
        cv2.putText(image, 'Stand straight to start, squat down to count reps', 
                    (10, frame_height - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display the image
        cv2.imshow('Squat Counter', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point (knee in this case)
    """
    a = np.array(a)  # First point (hip)
    b = np.array(b)  # Vertex point (knee)
    c = np.array(c)  # End point (ankle)
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Function to detect which leg is forward (lunging)
def detect_forward_leg(left_knee, right_knee, left_ankle, right_ankle):
    """
    Detect which leg is forward based on knee and ankle positions
    Returns 'left', 'right', or 'none'
    """
    # Calculate forward displacement (negative y means higher on screen/forward)
    left_forward_score = left_knee[1] - left_ankle[1]  # knee higher than ankle indicates forward lunge
    right_forward_score = right_knee[1] - right_ankle[1]
    
    # Also consider knee positions relative to each other
    knee_diff = abs(left_knee[1] - right_knee[1])
    
    # Threshold for detecting a lunge position
    forward_threshold = 0.05  # Adjust based on testing
    
    if knee_diff > forward_threshold:
        if left_knee[1] < right_knee[1]:  # Left knee is higher (forward)
            return 'left'
        else:  # Right knee is higher (forward)
            return 'right'
    
    return 'none'  # Standing position or unclear

# Initialize video capture
cap = cv2.VideoCapture(0)

# Lunge counter variables
left_counter = 0
right_counter = 0
total_counter = 0
left_stage = None
right_stage = None
current_leg = 'none'

# Lunge detection thresholds
LUNGE_DOWN_THRESHOLD = 90   # Lunge down position
LUNGE_UP_THRESHOLD = 160    # Standing position

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Initialize variables
        left_knee_angle = 0
        right_knee_angle = 0
        active_angle = 0
        active_leg = 'none'
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates for both legs (normalized)
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Calculate knee angles for both legs
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                
                # Detect which leg is forward (lunging)
                current_leg = detect_forward_leg(left_knee, right_knee, left_ankle, right_ankle)
                
                # Determine active leg and angle based on detection
                if current_leg == 'left':
                    active_leg = 'Left'
                    active_angle = left_knee_angle
                    active_stage = left_stage
                elif current_leg == 'right':
                    active_leg = 'Right'
                    active_angle = right_knee_angle
                    active_stage = right_stage
                else:
                    active_leg = 'Standing'
                    active_angle = min(left_knee_angle, right_knee_angle)  # Show the more bent knee
                    active_stage = None
                
                # Lunge counting logic for left leg
                if current_leg == 'left':
                    if left_knee_angle > LUNGE_UP_THRESHOLD:
                        left_stage = 'up'
                    if left_knee_angle < LUNGE_DOWN_THRESHOLD and left_stage == 'up':
                        left_stage = 'down'
                        left_counter += 1
                        total_counter += 1
                        print(f"Left lunge completed! Count: {left_counter}")
                
                # Lunge counting logic for right leg
                elif current_leg == 'right':
                    if right_knee_angle > LUNGE_UP_THRESHOLD:
                        right_stage = 'up'
                    if right_knee_angle < LUNGE_DOWN_THRESHOLD and right_stage == 'up':
                        right_stage = 'down'
                        right_counter += 1
                        total_counter += 1
                        print(f"Right lunge completed! Count: {right_counter}")
                
                # Convert knee coordinates to pixel coordinates for visualization
                left_knee_coords = tuple(np.multiply(left_knee, [frame_width, frame_height]).astype(int))
                right_knee_coords = tuple(np.multiply(right_knee, [frame_width, frame_height]).astype(int))
                
                # Draw angle indicators on both knees with different colors based on activity
                if current_leg == 'left':
                    # Highlight left knee as active
                    cv2.circle(image, left_knee_coords, 12, (0, 255, 0), -1)  # Green for active
                    cv2.circle(image, right_knee_coords, 8, (100, 100, 100), -1)  # Gray for inactive
                elif current_leg == 'right':
                    # Highlight right knee as active
                    cv2.circle(image, left_knee_coords, 8, (100, 100, 100), -1)  # Gray for inactive
                    cv2.circle(image, right_knee_coords, 12, (0, 255, 0), -1)  # Green for active
                else:
                    # Both knees neutral
                    cv2.circle(image, left_knee_coords, 8, (0, 255, 255), -1)  # Yellow for neutral
                    cv2.circle(image, right_knee_coords, 8, (0, 255, 255), -1)  # Yellow for neutral
                
                # Display angle text for both knees
                cv2.putText(image, f'{int(left_knee_angle)}°',
                           (left_knee_coords[0] - 50, left_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.putText(image, f'{int(right_knee_angle)}°',
                           (right_knee_coords[0] + 10, right_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
        except Exception as e:
            print("Error:", e)
        
        # Setup main status display
        cv2.rectangle(image, (10, 10), (450, 200), (50, 50, 50), -1)
        cv2.rectangle(image, (10, 10), (450, 200), (255, 255, 255), 2)
        
        # Display total lunges
        cv2.putText(image, 'TOTAL LUNGES', (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(total_counter), (20, 75), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.5, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display individual leg counts
        cv2.putText(image, f'Left: {left_counter}', (180, 50), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Right: {right_counter}', (180, 75), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Display active leg and stage
        cv2.putText(image, 'ACTIVE LEG', (300, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, active_leg, (300, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Display current stage
        current_stage = 'None'
        if current_leg == 'left' and left_stage:
            current_stage = left_stage.capitalize()
        elif current_leg == 'right' and right_stage:
            current_stage = right_stage.capitalize()
        elif current_leg == 'none':
            current_stage = 'Standing'
            
        cv2.putText(image, 'STAGE', (20, 110), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, current_stage, (20, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Display active knee angle
        cv2.putText(image, 'KNEE ANGLE', (200, 110), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'{int(active_angle)}°', (200, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Display both individual knee angles
        cv2.putText(image, f'L Knee: {int(left_knee_angle)}°', (20, 170), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f'R Knee: {int(right_knee_angle)}°', (180, 170), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Show lunge detection status
        detection_color = (0, 255, 0) if current_leg != 'none' else (100, 100, 100)
        cv2.putText(image, f'Detection: {current_leg.upper()}', (300, 170), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, detection_color, 1, cv2.LINE_AA)
        
        # Render pose detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Add instructions
        cv2.putText(image, 'Perform alternating lunges. Step forward and bend knee. Press Q to quit.', 
                    (10, frame_height - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display the image
        cv2.imshow('Lunge Counter with Auto-Detection', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Print final statistics
print(f"\nFinal Lunge Results:")
print(f"Total Lunges: {total_counter}")
print(f"Left Leg Lunges: {left_counter}")
print(f"Right Leg Lunges: {right_counter}")

cap.release()
cv2.destroyAllWindows()

Left lunge completed! Count: 1
Right lunge completed! Count: 1
Right lunge completed! Count: 2
Right lunge completed! Count: 3
Right lunge completed! Count: 4
Right lunge completed! Count: 5
Left lunge completed! Count: 2
Right lunge completed! Count: 6


KeyboardInterrupt: 

In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Vertex point
    c = np.array(c)  # End point
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to detect jumping jack stage
def detect_jumping_jack_stage(left_wrist, right_wrist, left_ankle, right_ankle, 
                            left_shoulder, right_shoulder, left_hip, right_hip):
    """
    Detect if person is in 'up' or 'down' stage of jumping jack
    Up stage: arms raised, legs spread apart
    Down stage: arms down, legs together
    """
    
    # Calculate arm spread (distance between wrists relative to shoulders)
    wrist_distance = calculate_distance(left_wrist, right_wrist)
    shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
    arm_spread_ratio = wrist_distance / shoulder_distance if shoulder_distance > 0 else 0
    
    # Calculate leg spread (distance between ankles relative to hips)
    ankle_distance = calculate_distance(left_ankle, right_ankle)
    hip_distance = calculate_distance(left_hip, right_hip)
    leg_spread_ratio = ankle_distance / hip_distance if hip_distance > 0 else 0
    
    # Calculate arm height (average wrist height relative to shoulders)
    avg_wrist_y = (left_wrist[1] + right_wrist[1]) / 2
    avg_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
    arm_height_ratio = (avg_shoulder_y - avg_wrist_y)  # Positive when arms are raised
    
    # Thresholds for detection (adjust based on testing)
    ARM_SPREAD_UP_THRESHOLD = 1.8    # Arms spread wide
    ARM_SPREAD_DOWN_THRESHOLD = 1.2  # Arms close to body
    LEG_SPREAD_UP_THRESHOLD = 1.8    # Legs spread apart
    LEG_SPREAD_DOWN_THRESHOLD = 1.2  # Legs close together
    ARM_HEIGHT_THRESHOLD = 0.1       # Arms raised above shoulders
    
    # Determine stage based on arm and leg positions
    arms_up = (arm_spread_ratio > ARM_SPREAD_UP_THRESHOLD and arm_height_ratio > ARM_HEIGHT_THRESHOLD)
    arms_down = (arm_spread_ratio < ARM_SPREAD_DOWN_THRESHOLD and arm_height_ratio < -ARM_HEIGHT_THRESHOLD)
    
    legs_apart = leg_spread_ratio > LEG_SPREAD_UP_THRESHOLD
    legs_together = leg_spread_ratio < LEG_SPREAD_DOWN_THRESHOLD
    
    if arms_up and legs_apart:
        return 'up', arm_spread_ratio, leg_spread_ratio, arm_height_ratio
    elif arms_down and legs_together:
        return 'down', arm_spread_ratio, leg_spread_ratio, arm_height_ratio
    else:
        return 'transition', arm_spread_ratio, leg_spread_ratio, arm_height_ratio

# Initialize video capture
cap = cv2.VideoCapture(0)

# Jumping jack counter variables
jj_counter = 0
jj_stage = None

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Initialize variables
        left_knee_angle = 0
        right_knee_angle = 0
        current_stage = 'none'
        arm_spread_ratio = 0
        leg_spread_ratio = 0
        arm_height_ratio = 0
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates for required landmarks (normalized)
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Calculate knee angles for both legs (hip-knee-ankle)
                left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                
                # Detect jumping jack stage
                current_stage, arm_spread_ratio, leg_spread_ratio, arm_height_ratio = detect_jumping_jack_stage(
                    left_wrist, right_wrist, left_ankle, right_ankle,
                    left_shoulder, right_shoulder, left_hip, right_hip
                )
                
                # Jumping jack counting logic
                if current_stage == 'up':
                    jj_stage = 'up'
                elif current_stage == 'down' and jj_stage == 'up':
                    jj_stage = 'down'
                    jj_counter += 1
                    print(f"Jumping jack completed! Count: {jj_counter}")
                
                # Convert coordinates to pixel coordinates for visualization
                left_wrist_coords = tuple(np.multiply(left_wrist, [frame_width, frame_height]).astype(int))
                right_wrist_coords = tuple(np.multiply(right_wrist, [frame_width, frame_height]).astype(int))
                left_ankle_coords = tuple(np.multiply(left_ankle, [frame_width, frame_height]).astype(int))
                right_ankle_coords = tuple(np.multiply(right_ankle, [frame_width, frame_height]).astype(int))
                left_knee_coords = tuple(np.multiply(left_knee, [frame_width, frame_height]).astype(int))
                right_knee_coords = tuple(np.multiply(right_knee, [frame_width, frame_height]).astype(int))
                
                # Draw indicators based on current stage
                if current_stage == 'up':
                    # Highlight key points in green when in up position
                    cv2.circle(image, left_wrist_coords, 8, (0, 255, 0), -1)
                    cv2.circle(image, right_wrist_coords, 8, (0, 255, 0), -1)
                    cv2.circle(image, left_ankle_coords, 8, (0, 255, 0), -1)
                    cv2.circle(image, right_ankle_coords, 8, (0, 255, 0), -1)
                elif current_stage == 'down':
                    # Highlight key points in blue when in down position
                    cv2.circle(image, left_wrist_coords, 8, (255, 0, 0), -1)
                    cv2.circle(image, right_wrist_coords, 8, (255, 0, 0), -1)
                    cv2.circle(image, left_ankle_coords, 8, (255, 0, 0), -1)
                    cv2.circle(image, right_ankle_coords, 8, (255, 0, 0), -1)
                else:
                    # Neutral color during transition
                    cv2.circle(image, left_wrist_coords, 6, (0, 255, 255), -1)
                    cv2.circle(image, right_wrist_coords, 6, (0, 255, 255), -1)
                    cv2.circle(image, left_ankle_coords, 6, (0, 255, 255), -1)
                    cv2.circle(image, right_ankle_coords, 6, (0, 255, 255), -1)
                
                # Draw knee angle indicators
                cv2.circle(image, left_knee_coords, 6, (255, 255, 0), -1)
                cv2.circle(image, right_knee_coords, 6, (255, 255, 0), -1)
                
                # Display knee angles
                cv2.putText(image, f'{int(left_knee_angle)}°',
                           (left_knee_coords[0] - 30, left_knee_coords[1] - 15),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                
                cv2.putText(image, f'{int(right_knee_angle)}°',
                           (right_knee_coords[0] + 10, right_knee_coords[1] - 15),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                
        except Exception as e:
            print("Error:", e)
        
        # Setup main status display
        cv2.rectangle(image, (10, 10), (500, 220), (50, 50, 50), -1)
        cv2.rectangle(image, (10, 10), (500, 220), (255, 255, 255), 2)
        
        # Display jumping jack count
        cv2.putText(image, 'JUMPING JACKS', (20, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(jj_counter), (20, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)
        
        # Display current stage
        stage_color = (0, 255, 0) if current_stage == 'up' else (255, 0, 0) if current_stage == 'down' else (0, 255, 255)
        cv2.putText(image, 'STAGE', (200, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, current_stage.upper(), (200, 75), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, stage_color, 2, cv2.LINE_AA)
        
        # Display knee angles
        cv2.putText(image, 'KNEE ANGLES', (350, 40), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'L: {int(left_knee_angle)}°', (350, 65), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'R: {int(right_knee_angle)}°', (350, 85), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Display detection metrics
        cv2.putText(image, 'DETECTION METRICS', (20, 115), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'Arm Spread: {arm_spread_ratio:.2f}', (20, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f'Leg Spread: {leg_spread_ratio:.2f}', (20, 160), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        cv2.putText(image, f'Arm Height: {arm_height_ratio:.2f}', (20, 180), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display current internal stage
        internal_stage = jj_stage if jj_stage else 'None'
        cv2.putText(image, f'Internal: {internal_stage}', (250, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1, cv2.LINE_AA)
        
        # Render pose detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Add instructions
        cv2.putText(image, 'Perform jumping jacks: Jump up with arms and legs spread, then back down. Press Q to quit.', 
                    (10, frame_height - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display the image
        cv2.imshow('Jumping Jack Counter', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Print final statistics
print(f"\nFinal Jumping Jack Results:")
print(f"Total Jumping Jacks: {jj_counter}")

cap.release()
cv2.destroyAllWindows()

Jumping jack completed! Count: 1
Jumping jack completed! Count: 2
Jumping jack completed! Count: 3


KeyboardInterrupt: 

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from collections import deque

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Vertex point
    c = np.array(c)  # End point
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to detect push-up stage with improved logic
def detect_pushup_stage(left_elbow_angle, right_elbow_angle, body_straightness, 
                       left_shoulder, right_shoulder, left_hip, right_hip):
    """
    Detect if person is in 'up' or 'down' stage of push-up
    Down stage: elbows bent (smaller angles), body straight and low
    Up stage: elbows extended (larger angles), body straight and high
    """
    
    # Average elbow angle for more stable detection
    avg_elbow_angle = (left_elbow_angle + right_elbow_angle) / 2
    
    # Calculate body height (average shoulder height)
    avg_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
    avg_hip_y = (left_hip[1] + right_hip[1]) / 2
    
    # Thresholds for push-up detection
    DOWN_ELBOW_THRESHOLD = 90   # Elbows bent position
    UP_ELBOW_THRESHOLD = 140    # Elbows extended position
    BODY_STRAIGHT_THRESHOLD = 0.15  # Body alignment threshold
    
    # Check body straightness (shoulders-hips alignment)
    is_body_straight = body_straightness < BODY_STRAIGHT_THRESHOLD
    
    # Determine stage based on elbow angles and body position
    if avg_elbow_angle < DOWN_ELBOW_THRESHOLD and is_body_straight:
        return 'down'
    elif avg_elbow_angle > UP_ELBOW_THRESHOLD and is_body_straight:
        return 'up'
    else:
        return 'transition'

# Function to calculate body straightness
def calculate_body_straightness(left_shoulder, right_shoulder, left_hip, right_hip, 
                              left_knee, right_knee, left_ankle, right_ankle):
    """
    Calculate how straight the body is from shoulders to ankles
    Returns a value where lower means straighter
    """
    # Calculate midpoints
    shoulder_mid = [(left_shoulder[0] + right_shoulder[0]) / 2, 
                   (left_shoulder[1] + right_shoulder[1]) / 2]
    hip_mid = [(left_hip[0] + right_hip[0]) / 2, 
               (left_hip[1] + right_hip[1]) / 2]
    knee_mid = [(left_knee[0] + right_knee[0]) / 2, 
                (left_knee[1] + right_knee[1]) / 2]
    ankle_mid = [(left_ankle[0] + right_ankle[0]) / 2, 
                 (left_ankle[1] + right_ankle[1]) / 2]
    
    # Calculate angles between body segments
    shoulder_hip_knee_angle = calculate_angle(shoulder_mid, hip_mid, knee_mid)
    hip_knee_ankle_angle = calculate_angle(hip_mid, knee_mid, ankle_mid)
    
    # Deviation from straight line (180 degrees)
    straightness_score = abs(180 - shoulder_hip_knee_angle) + abs(180 - hip_knee_ankle_angle)
    
    return straightness_score / 360.0  # Normalize to 0-1 range

# Smoothing class for stable detection
class SmoothingFilter:
    def __init__(self, window_size=5):
        self.window_size = window_size
        self.values = deque(maxlen=window_size)
    
    def add_value(self, value):
        self.values.append(value)
    
    def get_smoothed_value(self):
        if len(self.values) == 0:
            return 0
        return sum(self.values) / len(self.values)
    
    def get_trend(self):
        if len(self.values) < 2:
            return 'stable'
        recent = sum(list(self.values)[-3:]) / min(3, len(self.values))
        older = sum(list(self.values)[:-3]) / max(1, len(self.values) - 3)
        if recent > older + 5:
            return 'increasing'
        elif recent < older - 5:
            return 'decreasing'
        else:
            return 'stable'

# Initialize video capture
cap = cv2.VideoCapture(0)

# Push-up counter variables
pushup_counter = 0
pushup_stage = None
last_stage = None

# Smoothing filters
left_elbow_filter = SmoothingFilter(window_size=7)
right_elbow_filter = SmoothingFilter(window_size=7)
body_straightness_filter = SmoothingFilter(window_size=5)

# Stage transition tracking for robust counting
stage_history = deque(maxlen=10)
min_stage_duration = 3  # Minimum frames to confirm stage change

# Set up Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Make detection
        results = pose.process(image)
        
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Initialize variables
        left_elbow_angle = 0
        right_elbow_angle = 0
        left_hip_angle = 0
        right_hip_angle = 0
        body_straightness = 1.0
        current_stage = 'none'
        smoothed_left_elbow = 0
        smoothed_right_elbow = 0
        avg_elbow_angle = 0
        
        # Extract landmarks
        try:
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates for required landmarks (normalized)
                left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                
                left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Calculate joint angles
                # Elbow angles (shoulder-elbow-wrist)
                left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
                right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
                
                # Hip angles (shoulder-hip-knee) for body posture
                left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
                right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
                
                # Calculate body straightness
                body_straightness = calculate_body_straightness(
                    left_shoulder, right_shoulder, left_hip, right_hip,
                    left_knee, right_knee, left_ankle, right_ankle
                )
                
                # Apply smoothing to elbow angles
                left_elbow_filter.add_value(left_elbow_angle)
                right_elbow_filter.add_value(right_elbow_angle)
                body_straightness_filter.add_value(body_straightness)
                
                smoothed_left_elbow = left_elbow_filter.get_smoothed_value()
                smoothed_right_elbow = right_elbow_filter.get_smoothed_value()
                smoothed_body_straightness = body_straightness_filter.get_smoothed_value()
                avg_elbow_angle = (smoothed_left_elbow + smoothed_right_elbow) / 2
                
                # Detect push-up stage using smoothed values
                current_stage = detect_pushup_stage(
                    smoothed_left_elbow, smoothed_right_elbow, 
                    smoothed_body_straightness, left_shoulder, right_shoulder, 
                    left_hip, right_hip
                )
                
                # Add to stage history for robust detection
                stage_history.append(current_stage)
                
                # Count stages only if consistent over multiple frames
                if len(stage_history) >= min_stage_duration:
                    recent_stages = list(stage_history)[-min_stage_duration:]
                    if all(stage == current_stage for stage in recent_stages):
                        # Stage is consistent, update push-up logic
                        if current_stage == 'up' and pushup_stage != 'up':
                            pushup_stage = 'up'
                        elif current_stage == 'down' and pushup_stage == 'up':
                            pushup_stage = 'down'
                            pushup_counter += 1
                            print(f"Push-up completed! Count: {pushup_counter}")
                
                # Convert coordinates to pixel coordinates for visualization
                left_elbow_coords = tuple(np.multiply(left_elbow, [frame_width, frame_height]).astype(int))
                right_elbow_coords = tuple(np.multiply(right_elbow, [frame_width, frame_height]).astype(int))
                left_wrist_coords = tuple(np.multiply(left_wrist, [frame_width, frame_height]).astype(int))
                right_wrist_coords = tuple(np.multiply(right_wrist, [frame_width, frame_height]).astype(int))
                left_shoulder_coords = tuple(np.multiply(left_shoulder, [frame_width, frame_height]).astype(int))
                right_shoulder_coords = tuple(np.multiply(right_shoulder, [frame_width, frame_height]).astype(int))
                
                # Draw indicators based on current stage
                if current_stage == 'down':
                    # Highlight key points in red when in down position
                    cv2.circle(image, left_elbow_coords, 10, (0, 0, 255), -1)
                    cv2.circle(image, right_elbow_coords, 10, (0, 0, 255), -1)
                    cv2.circle(image, left_wrist_coords, 6, (0, 0, 255), -1)
                    cv2.circle(image, right_wrist_coords, 6, (0, 0, 255), -1)
                elif current_stage == 'up':
                    # Highlight key points in green when in up position
                    cv2.circle(image, left_elbow_coords, 10, (0, 255, 0), -1)
                    cv2.circle(image, right_elbow_coords, 10, (0, 255, 0), -1)
                    cv2.circle(image, left_wrist_coords, 6, (0, 255, 0), -1)
                    cv2.circle(image, right_wrist_coords, 6, (0, 255, 0), -1)
                else:
                    # Neutral color during transition
                    cv2.circle(image, left_elbow_coords, 8, (0, 255, 255), -1)
                    cv2.circle(image, right_elbow_coords, 8, (0, 255, 255), -1)
                    cv2.circle(image, left_wrist_coords, 4, (0, 255, 255), -1)
                    cv2.circle(image, right_wrist_coords, 4, (0, 255, 255), -1)
                
                # Draw shoulder indicators
                cv2.circle(image, left_shoulder_coords, 6, (255, 255, 0), -1)
                cv2.circle(image, right_shoulder_coords, 6, (255, 255, 0), -1)
                
                # Display elbow angles
                cv2.putText(image, f'{int(smoothed_left_elbow)}°',
                           (left_elbow_coords[0] - 40, left_elbow_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.putText(image, f'{int(smoothed_right_elbow)}°',
                           (right_elbow_coords[0] + 10, right_elbow_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
        except Exception as e:
            print("Error:", e)
        
        # Setup main status display
        cv2.rectangle(image, (10, 10), (550, 280), (50, 50, 50), -1)
        cv2.rectangle(image, (10, 10), (550, 280), (255, 255, 255), 2)
        
        # Display push-up count
        cv2.putText(image, 'PUSH-UPS', (20, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, str(pushup_counter), (20, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2.5, (0, 255, 0), 3, cv2.LINE_AA)
        
        # Display current stage
        stage_color = (0, 255, 0) if current_stage == 'up' else (0, 0, 255) if current_stage == 'down' else (0, 255, 255)
        cv2.putText(image, 'STAGE', (200, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, current_stage.upper(), (200, 80), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, stage_color, 2, cv2.LINE_AA)
        
        # Display elbow angles
        cv2.putText(image, 'ELBOW ANGLES', (350, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'L: {int(smoothed_left_elbow)}°', (350, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'R: {int(smoothed_right_elbow)}°', (350, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'Avg: {int(avg_elbow_angle)}°', (450, 82), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Display hip angles
        cv2.putText(image, 'HIP ANGLES', (20, 130), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.putText(image, f'L: {int(left_hip_angle)}°', (20, 155), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(image, f'R: {int(right_hip_angle)}°', (120, 155), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
        
        # Display body metrics
        cv2.putText(image, 'BODY METRICS', (250, 130), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
        straightness_color = (0, 255, 0) if smoothed_body_straightness < 0.15 else (255, 255, 0) if smoothed_body_straightness < 0.3 else (0, 0, 255)
        cv2.putText(image, f'Straightness: {smoothed_body_straightness:.2f}', (250, 155), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, straightness_color, 1, cv2.LINE_AA)
        
        # Display internal stage for debugging
        internal_stage = pushup_stage if pushup_stage else 'None'
        cv2.putText(image, f'Internal: {internal_stage}', (250, 175), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1, cv2.LINE_AA)
        
        # Display angle trends
        left_trend = left_elbow_filter.get_trend()
        right_trend = right_elbow_filter.get_trend()
        cv2.putText(image, f'L Trend: {left_trend}', (20, 200), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1, cv2.LINE_AA)
        cv2.putText(image, f'R Trend: {right_trend}', (20, 220), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (180, 180, 180), 1, cv2.LINE_AA)
        
        # Display smoothing status
        cv2.putText(image, f'Smooth Frames: {len(stage_history)}/{min_stage_duration}', (20, 250), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.4, (150, 150, 150), 1, cv2.LINE_AA)
        
        # Render pose detections
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
            )
        
        # Add instructions
        cv2.putText(image, 'Perform push-ups: Keep body straight, lower chest to ground, push back up. Press Q to quit.', 
                    (10, frame_height - 20), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
        
        # Display the image
        cv2.imshow('Push-up Counter with Smoothing', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

# Print final statistics
print(f"\nFinal Push-up Results:")
print(f"Total Push-ups: {pushup_counter}")

cap.release()
cv2.destroyAllWindows()

Push-up completed! Count: 1
Push-up completed! Count: 2
Push-up completed! Count: 3


KeyboardInterrupt: 

In [10]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize MediaPipe
mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

def calculate_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point (knee in this case)
    """
    a = np.array(a)  # First point (hip)
    b = np.array(b)  # Vertex point (knee)
    c = np.array(c)  # End point (ankle)
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

def is_finger_extended(tip, pip, mcp, wrist):
    """
    Check if a finger is extended by comparing distances
    More robust approach that works from multiple angles
    """
    # Distance from wrist to tip
    tip_dist = calculate_distance(tip, wrist)
    # Distance from wrist to middle joint
    pip_dist = calculate_distance(pip, wrist)
    # Distance from wrist to base
    mcp_dist = calculate_distance(mcp, wrist)
    
    # Finger is extended if tip is farther from wrist than middle joint
    # and middle joint is farther than base joint
    return tip_dist > pip_dist and pip_dist > mcp_dist * 0.8

def is_thumb_extended(thumb_tip, thumb_ip, thumb_mcp, wrist):
    """
    Special case for thumb detection - more lenient
    """
    # Distance from wrist to thumb tip
    tip_dist = calculate_distance(thumb_tip, wrist)
    # Distance from wrist to thumb middle joint
    ip_dist = calculate_distance(thumb_ip, wrist)
    # Distance from wrist to thumb base
    mcp_dist = calculate_distance(thumb_mcp, wrist)
    
    # Thumb is extended if tip is farther from wrist than middle joint
    return tip_dist > ip_dist * 0.9 and tip_dist > mcp_dist * 0.7

def detect_thumbs_up_robust(landmarks):
    """
    Enhanced thumbs up detection that works from multiple angles
    """
    # Landmark indices
    wrist = 0
    thumb_tip = 4
    thumb_ip = 3
    thumb_mcp = 2
    index_tip = 8
    index_pip = 6
    index_mcp = 5
    middle_tip = 12
    middle_pip = 10
    middle_mcp = 9
    ring_tip = 16
    ring_pip = 14
    ring_mcp = 13
    pinky_tip = 20
    pinky_pip = 18
    pinky_mcp = 17
    
    # Get coordinates
    wrist_coords = [landmarks[wrist].x, landmarks[wrist].y]
    thumb_tip_coords = [landmarks[thumb_tip].x, landmarks[thumb_tip].y]
    thumb_ip_coords = [landmarks[thumb_ip].x, landmarks[thumb_ip].y]
    thumb_mcp_coords = [landmarks[thumb_mcp].x, landmarks[thumb_mcp].y]
    
    index_tip_coords = [landmarks[index_tip].x, landmarks[index_tip].y]
    index_pip_coords = [landmarks[index_pip].x, landmarks[index_pip].y]
    index_mcp_coords = [landmarks[index_mcp].x, landmarks[index_mcp].y]
    
    middle_tip_coords = [landmarks[middle_tip].x, landmarks[middle_tip].y]
    middle_pip_coords = [landmarks[middle_pip].x, landmarks[middle_pip].y]
    middle_mcp_coords = [landmarks[middle_mcp].x, landmarks[middle_mcp].y]
    
    ring_tip_coords = [landmarks[ring_tip].x, landmarks[ring_tip].y]
    ring_pip_coords = [landmarks[ring_pip].x, landmarks[ring_pip].y]
    ring_mcp_coords = [landmarks[ring_mcp].x, landmarks[ring_mcp].y]
    
    pinky_tip_coords = [landmarks[pinky_tip].x, landmarks[pinky_tip].y]
    pinky_pip_coords = [landmarks[pinky_pip].x, landmarks[pinky_pip].y]
    pinky_mcp_coords = [landmarks[pinky_mcp].x, landmarks[pinky_mcp].y]
    
    # Check if thumb is extended
    thumb_extended = is_thumb_extended(thumb_tip_coords, thumb_ip_coords, thumb_mcp_coords, wrist_coords)
    
    # Check if other fingers are NOT extended (folded)
    index_folded = not is_finger_extended(index_tip_coords, index_pip_coords, index_mcp_coords, wrist_coords)
    middle_folded = not is_finger_extended(middle_tip_coords, middle_pip_coords, middle_mcp_coords, wrist_coords)
    ring_folded = not is_finger_extended(ring_tip_coords, ring_pip_coords, ring_mcp_coords, wrist_coords)
    pinky_folded = not is_finger_extended(pinky_tip_coords, pinky_pip_coords, pinky_mcp_coords, wrist_coords)
    
    # Additional check: thumb should be reasonably separated from palm center
    palm_center = [
        (landmarks[0].x + landmarks[5].x + landmarks[9].x + landmarks[13].x + landmarks[17].x) / 5,
        (landmarks[0].y + landmarks[5].y + landmarks[9].y + landmarks[13].y + landmarks[17].y) / 5
    ]
    thumb_separated = calculate_distance(thumb_tip_coords, palm_center) > 0.08
    
    # Count how many conditions are met (more flexible approach)
    conditions_met = sum([
        thumb_extended,
        thumb_separated,
        index_folded,
        middle_folded,
        ring_folded,
        pinky_folded
    ])
    
    # Require at least 4 out of 6 conditions to be met
    return conditions_met >= 4

def detect_peace_sign_robust(landmarks):
    """
    Enhanced peace sign detection that works from multiple angles
    """
    # Landmark indices
    wrist = 0
    thumb_tip = 4
    thumb_pip = 3
    thumb_mcp = 2
    index_tip = 8
    index_pip = 6
    index_mcp = 5
    middle_tip = 12
    middle_pip = 10
    middle_mcp = 9
    ring_tip = 16
    ring_pip = 14
    ring_mcp = 13
    pinky_tip = 20
    pinky_pip = 18
    pinky_mcp = 17
    
    # Get coordinates
    wrist_coords = [landmarks[wrist].x, landmarks[wrist].y]
    
    index_tip_coords = [landmarks[index_tip].x, landmarks[index_tip].y]
    index_pip_coords = [landmarks[index_pip].x, landmarks[index_pip].y]
    index_mcp_coords = [landmarks[index_mcp].x, landmarks[index_mcp].y]
    
    middle_tip_coords = [landmarks[middle_tip].x, landmarks[middle_tip].y]
    middle_pip_coords = [landmarks[middle_pip].x, landmarks[middle_pip].y]
    middle_mcp_coords = [landmarks[middle_mcp].x, landmarks[middle_mcp].y]
    
    thumb_tip_coords = [landmarks[thumb_tip].x, landmarks[thumb_tip].y]
    thumb_pip_coords = [landmarks[thumb_pip].x, landmarks[thumb_pip].y]
    thumb_mcp_coords = [landmarks[thumb_mcp].x, landmarks[thumb_mcp].y]
    
    ring_tip_coords = [landmarks[ring_tip].x, landmarks[ring_tip].y]
    ring_pip_coords = [landmarks[ring_pip].x, landmarks[ring_pip].y]
    ring_mcp_coords = [landmarks[ring_mcp].x, landmarks[ring_mcp].y]
    
    pinky_tip_coords = [landmarks[pinky_tip].x, landmarks[pinky_tip].y]
    pinky_pip_coords = [landmarks[pinky_pip].x, landmarks[pinky_pip].y]
    pinky_mcp_coords = [landmarks[pinky_mcp].x, landmarks[pinky_mcp].y]
    
    # Check if index and middle fingers are extended
    index_extended = is_finger_extended(index_tip_coords, index_pip_coords, index_mcp_coords, wrist_coords)
    middle_extended = is_finger_extended(middle_tip_coords, middle_pip_coords, middle_mcp_coords, wrist_coords)
    
    # Check if thumb, ring, and pinky are NOT extended (folded)
    thumb_folded = not is_thumb_extended(thumb_tip_coords, thumb_pip_coords, thumb_mcp_coords, wrist_coords)
    ring_folded = not is_finger_extended(ring_tip_coords, ring_pip_coords, ring_mcp_coords, wrist_coords)
    pinky_folded = not is_finger_extended(pinky_tip_coords, pinky_pip_coords, pinky_mcp_coords, wrist_coords)
    
    # Check that index and middle fingers are reasonably separated
    fingers_separated = calculate_distance(index_tip_coords, middle_tip_coords) > 0.04
    
    # Alternative separation check using angles
    palm_center = [
        (landmarks[0].x + landmarks[5].x + landmarks[9].x + landmarks[13].x + landmarks[17].x) / 5,
        (landmarks[0].y + landmarks[5].y + landmarks[9].y + landmarks[13].y + landmarks[17].y) / 5
    ]
    
    # Check if both fingers are extended away from palm
    index_from_center = calculate_distance(index_tip_coords, palm_center) > 0.12
    middle_from_center = calculate_distance(middle_tip_coords, palm_center) > 0.12
    
    # Count conditions met
    conditions_met = sum([
        index_extended,
        middle_extended,
        fingers_separated,
        index_from_center,
        middle_from_center,
        thumb_folded,
        ring_folded,
        pinky_folded
    ])
    
    # Require at least 5 out of 8 conditions to be met
    return conditions_met >= 5

def format_time(seconds):
    """Format seconds into HH:MM:SS format"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    return f"{hours:02d}:{minutes:02d}:{secs:02d}"

# Initialize video capture
cap = cv2.VideoCapture(0)

# Timer variables
timer_running = False
start_time = 0
elapsed_time = 0
total_elapsed = 0

# Gesture detection variables with improved settings
last_gesture_time = 0
gesture_cooldown = 1.0  # Reduced cooldown for better responsiveness
gesture_hold_time = 0.2  # Reduced hold time for quicker detection
current_gesture = None
gesture_start_time = 0
gesture_confirmed = False

# Gesture stability tracking
gesture_history = []
gesture_history_size = 5  # Track last 5 frames

# Squat counter variables
squat_counter = 0
squat_stage = None

print("Enhanced Fitness Tracker Started!")
print("👍 Thumbs up: Resume timer")
print("✌️ Peace sign: Pause timer")
print("🏋️ Perform squats to count reps (ONLY WHEN TIMER IS RUNNING)")
print("Press 'q' to quit, 'r' to reset both timer and counter")

# Set up MediaPipe instances with improved settings
with mp_hands.Hands(
    model_complexity=1,  # Increased complexity for better detection
    min_detection_confidence=0.6,  # Lowered for easier detection
    min_tracking_confidence=0.6,   # Lowered for better tracking
    max_num_hands=2  # Allow both hands for more flexibility
) as hands, mp_pose.Pose(
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5
) as pose:
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            print("Failed to grab frame")
            break
            
        # Get frame dimensions
        frame_height, frame_width = frame.shape[:2]
        
        # Flip frame horizontally for mirror effect
        frame = cv2.flip(frame, 1)
        
        # Convert BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        # Process the frame for both hand and pose detection
        hand_results = hands.process(image)
        pose_results = pose.process(image)
        
        # Convert back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        current_time = time.time()
        detected_gesture = None
        
        # ===== ENHANCED HAND GESTURE PROCESSING =====
        if hand_results.multi_hand_landmarks and hand_results.multi_handedness:
            gesture_votes = {"thumbs_up": 0, "peace_sign": 0}
            
            for hand_landmarks, handedness in zip(hand_results.multi_hand_landmarks, hand_results.multi_handedness):
                # Get hand label
                hand_label = handedness.classification[0].label
                
                # Detect gestures with enhanced functions
                thumbs_up = detect_thumbs_up_robust(hand_landmarks.landmark)
                peace_sign = detect_peace_sign_robust(hand_landmarks.landmark)
                
                # Vote for gestures
                if thumbs_up and not peace_sign:
                    gesture_votes["thumbs_up"] += 1
                elif peace_sign and not thumbs_up:
                    gesture_votes["peace_sign"] += 1
                
                # Get hand center for visual feedback
                wrist = hand_landmarks.landmark[0]
                center_coords = tuple(np.multiply([wrist.x, wrist.y], 
                                                [frame_width, frame_height]).astype(int))
                
                # Visual feedback for hand gestures
                if thumbs_up and not peace_sign:
                    cv2.circle(image, center_coords, 30, (0, 255, 0), 4)
                    cv2.putText(image, f"👍 {hand_label}", 
                               (center_coords[0] - 70, center_coords[1] - 40),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                elif peace_sign and not thumbs_up:
                    cv2.circle(image, center_coords, 30, (0, 0, 255), 4)
                    cv2.putText(image, f"✌️ {hand_label}", 
                               (center_coords[0] - 70, center_coords[1] - 40),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
                elif thumbs_up or peace_sign:
                    # Show partial detection
                    cv2.circle(image, center_coords, 20, (255, 255, 0), 2)
                    cv2.putText(image, f"? {hand_label}", 
                               (center_coords[0] - 40, center_coords[1] - 30),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
                
                # Draw hand landmarks
                mp_drawing.draw_landmarks(
                    image, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255, 0, 255), thickness=2)
                )
            
            # Determine winning gesture
            if gesture_votes["thumbs_up"] > 0:
                detected_gesture = "thumbs_up"
            elif gesture_votes["peace_sign"] > 0:
                detected_gesture = "peace_sign"
        
        # ===== GESTURE HISTORY TRACKING FOR STABILITY =====
        gesture_history.append(detected_gesture)
        if len(gesture_history) > gesture_history_size:
            gesture_history.pop(0)
        
        # Use majority vote from recent history for stability
        if len(gesture_history) >= 3:
            gesture_counts = {}
            for g in gesture_history:
                if g:
                    gesture_counts[g] = gesture_counts.get(g, 0) + 1
            
            if gesture_counts:
                stable_gesture = max(gesture_counts.items(), key=lambda x: x[1])[0]
                if gesture_counts[stable_gesture] >= 2:  # At least 2 out of last few frames
                    detected_gesture = stable_gesture
        
        # ===== POSE PROCESSING FOR SQUATS (ONLY WHEN TIMER IS RUNNING) =====
        try:
            if pose_results.pose_landmarks:
                landmarks = pose_results.pose_landmarks.landmark
                
                # Get coordinates for left leg (normalized)
                left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                           landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                            landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                             landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                
                # Get coordinates for right leg (normalized)
                right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                            landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                             landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                              landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                
                # Calculate angles for both knees
                left_angle = calculate_angle(left_hip, left_knee, left_ankle)
                right_angle = calculate_angle(right_hip, right_knee, right_ankle)
                
                # Use average of both knee angles for squat detection
                avg_angle = (left_angle + right_angle) / 2
                
                # Convert knee coordinates to pixel coordinates for display
                left_knee_coords = tuple(np.multiply(left_knee, [frame_width, frame_height]).astype(int))
                right_knee_coords = tuple(np.multiply(right_knee, [frame_width, frame_height]).astype(int))
                
                # Draw circles and put angle text at both knees
                # Color changes based on timer status
                knee_color = (0, 255, 0) if timer_running else (128, 128, 128)
                cv2.circle(image, left_knee_coords, 8, knee_color, -1)
                cv2.putText(image, f'{int(left_angle)}°',
                           (left_knee_coords[0] - 40, left_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                cv2.circle(image, right_knee_coords, 8, knee_color, -1)
                cv2.putText(image, f'{int(right_angle)}°',
                           (right_knee_coords[0] + 15, right_knee_coords[1] - 20),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
                
                # *** MODIFIED: Squat counter logic ONLY when timer is running ***
                if timer_running:
                    if avg_angle > 160:
                        squat_stage = 'up'
                    
                    if avg_angle < 90 and squat_stage == 'up':
                        squat_stage = 'down'
                        squat_counter += 1
                        print(f"Squat count: {squat_counter} (Timer: {format_time(elapsed_time)})")
                else:
                    # When timer is paused, don't update squat stage but keep tracking for display
                    if avg_angle > 160:
                        squat_stage = 'up (paused)'
                    elif avg_angle < 90:
                        squat_stage = 'down (paused)'
                
        except Exception as e:
            print("Pose detection error:", e)
        
        # ===== ENHANCED GESTURE TIMER LOGIC =====
        if detected_gesture:
            if current_gesture != detected_gesture:
                current_gesture = detected_gesture
                gesture_start_time = current_time
                gesture_confirmed = False
            elif (current_time - gesture_start_time) >= gesture_hold_time and not gesture_confirmed:
                gesture_confirmed = True
                
                if (current_time - last_gesture_time) > gesture_cooldown:
                    if detected_gesture == "thumbs_up" and not timer_running:
                        timer_running = True
                        start_time = current_time
                        last_gesture_time = current_time
                        gesture_history.clear()  # Clear history after successful gesture
                        print("Timer RESUMED (👍) - Squats will now be counted!")
                    elif detected_gesture == "peace_sign" and timer_running:
                        timer_running = False
                        total_elapsed += (current_time - start_time)
                        last_gesture_time = current_time
                        gesture_history.clear()  # Clear history after successful gesture
                        print(f"Timer PAUSED (✌️) at: {format_time(total_elapsed)} - Squats will NOT be counted while paused")
        else:
            current_gesture = None
            gesture_confirmed = False
        
        # Calculate current elapsed time
        if timer_running:
            elapsed_time = total_elapsed + (current_time - start_time)
        else:
            elapsed_time = total_elapsed
        
        # ===== ENHANCED DISPLAY UI =====
        # Create main status box with better visibility
        cv2.rectangle(image, (0, 0), (650, 250), (40, 40, 40), -1)
        cv2.rectangle(image, (0, 0), (650, 250), (255, 255, 255), 2)
        
        # Timer section
        cv2.putText(image, 'TIMER', (20, 35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        timer_text = format_time(elapsed_time)
        cv2.putText(image, timer_text, (20, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.4, (0, 255, 255), 3)
        
        # Timer status
        status = "RUNNING" if timer_running else "PAUSED"
        status_color = (0, 255, 0) if timer_running else (0, 0, 255)
        cv2.putText(image, status, (20, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, status_color, 2)
        
        # Squat counter section
        cv2.putText(image, 'SQUATS', (250, 35), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        
        # Display squat counter with different color based on timer status
        squat_color = (255, 255, 255) if timer_running else (128, 128, 128)
        cv2.putText(image, str(squat_counter), (250, 70), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1.4, squat_color, 3)
        
        # Squat stage with timer status indication
        if squat_stage:
            stage_color = (255, 255, 255) if timer_running else (128, 128, 128)
            cv2.putText(image, squat_stage.upper(), (250, 95), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, stage_color, 2)
        
        # Squat counting status indicator
        counting_status = "COUNTING" if timer_running else "NOT COUNTING"
        counting_color = (0, 255, 0) if timer_running else (255, 0, 0)
        cv2.putText(image, counting_status, (250, 115), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, counting_color, 2)
        
        # Average angle display
        try:
            angle_color = (255, 255, 255) if timer_running else (128, 128, 128)
            cv2.putText(image, f'AVG: {int(avg_angle)}°', (450, 70), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, angle_color, 2)
        except:
            pass
        
        # Gesture detection status
        gesture_status = "None"
        gesture_color = (128, 128, 128)
        if detected_gesture == "thumbs_up":
            gesture_status = "👍 THUMBS UP"
            gesture_color = (0, 255, 0)
        elif detected_gesture == "peace_sign":
            gesture_status = "✌️ PEACE SIGN"
            gesture_color = (0, 0, 255)
        
        cv2.putText(image, 'GESTURE:', (20, 135), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        cv2.putText(image, gesture_status, (20, 155), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, gesture_color, 2)
        
        # Gesture hold progress
        if current_gesture and not gesture_confirmed:
            progress = min((current_time - gesture_start_time) / gesture_hold_time, 1.0)
            cv2.putText(image, f'Hold: {progress:.0%}', (20, 175), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)
        
        # Cooldown indicator
        if (current_time - last_gesture_time) < gesture_cooldown:
            cooldown_remaining = gesture_cooldown - (current_time - last_gesture_time)
            cv2.putText(image, f'Cooldown: {cooldown_remaining:.1f}s', 
                        (20, 195),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 100, 0), 1)
        
        # Timer-controlled squat counting notice
        notice_color = (0, 255, 0) if timer_running else (255, 100, 0)
        notice_text = "✓ Squats counting active" if timer_running else "⚠ Start timer to count squats"
        cv2.putText(image, notice_text, (20, 220), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, notice_color, 2)
        
        # Enhanced instructions
        instructions = [
            "👍 Thumbs up: Resume timer & start counting squats",
            "✌️ Peace: Pause timer & stop counting squats",
            "🏋️ Squats only counted when timer is RUNNING",
            "Press 'R' to reset all, 'Q' to quit"
        ]
        
        y_offset = frame_height - 85
        for instruction in instructions:
            cv2.putText(image, instruction, (10, y_offset), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            y_offset += 18
        
        # Render pose landmarks
        if pose_results.pose_landmarks:
            # Change pose landmark colors based on timer status
            pose_color = (245, 117, 66) if timer_running else (100, 100, 100)
            connection_color = (245, 66, 230) if timer_running else (120, 120, 120)
            
            mp_drawing.draw_landmarks(
                image, pose_results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=pose_color, thickness=2, circle_radius=2),
                mp_drawing.DrawingSpec(color=connection_color, thickness=2, circle_radius=2)
            )
        
        # Show frame
        cv2.imshow('Enhanced Fitness Tracker - Timer-Controlled Squat Counter', image)
        
        # Handle keyboard input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
        elif key == ord('r') or key == ord('R'):
            # Reset everything
            timer_running = False
            start_time = 0
            elapsed_time = 0
            total_elapsed = 0
            squat_counter = 0
            squat_stage = None
            gesture_history.clear()
            print("Timer and squat counter RESET")

# Cleanup
cap.release()
cv2.destroyAllWindows()

Enhanced Fitness Tracker Started!
👍 Thumbs up: Resume timer
✌️ Peace sign: Pause timer
🏋️ Perform squats to count reps (ONLY WHEN TIMER IS RUNNING)
Press 'q' to quit, 'r' to reset both timer and counter
Timer RESUMED (👍) - Squats will now be counted!
Timer PAUSED (✌️) at: 00:00:07 - Squats will NOT be counted while paused
Timer RESUMED (👍) - Squats will now be counted!
Timer PAUSED (✌️) at: 00:00:13 - Squats will NOT be counted while paused
Timer RESUMED (👍) - Squats will now be counted!
Squat count: 1 (Timer: 00:00:15)
Squat count: 2 (Timer: 00:00:16)
Timer PAUSED (✌️) at: 00:00:17 - Squats will NOT be counted while paused
Timer RESUMED (👍) - Squats will now be counted!


KeyboardInterrupt: 

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import time
from collections import deque
from typing import Tuple, Optional, List

# Initialize MediaPipe
mp_pose = mp.solutions.pose
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

class GestureTimer:
    """
    Robust gesture recognition system for timer control.
    Supports thumbs up (resume) and peace sign (pause) gestures.
    """
    
    def __init__(self, confidence_threshold=0.8, smoothing_frames=5):
        """
        Initialize the gesture timer.
        
        Args:
            confidence_threshold: Minimum confidence for gesture detection
            smoothing_frames: Number of frames for gesture smoothing
        """
        self.confidence_threshold = confidence_threshold
        self.smoothing_frames = smoothing_frames
        
        # Timer state
        self.is_running = False
        self.start_time = None
        self.elapsed_time = 0.0
        self.pause_time = None
        
        # Gesture detection smoothing
        self.gesture_history = deque(maxlen=smoothing_frames)
        self.last_gesture = None
        self.gesture_change_time = time.time()
        
        # Initialize MediaPipe hands
        self.hands = mp_hands.Hands(
            static_image_mode=False,
            max_num_hands=2,
            min_detection_confidence=0.7,
            min_tracking_confidence=0.5
        )
    
    def get_hand_landmarks(self, image):
        """Extract hand landmarks from image."""
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = self.hands.process(rgb_image)
        return results.multi_hand_landmarks, results.multi_handedness
    
    def calculate_finger_angles(self, landmarks):
        """
        Calculate angles for each finger to determine hand pose.
        Returns angles for thumb, index, middle, ring, pinky.
        """
        def get_angle(p1, p2, p3):
            """Calculate angle between three points."""
            v1 = np.array([p1.x - p2.x, p1.y - p2.y])
            v2 = np.array([p3.x - p2.x, p3.y - p2.y])
            
            cosine = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            cosine = np.clip(cosine, -1.0, 1.0)
            angle = np.arccos(cosine)
            return np.degrees(angle)
        
        # Landmark indices for finger joints
        finger_joints = [
            [1, 2, 3],    # Thumb
            [5, 6, 7],    # Index
            [9, 10, 11],  # Middle
            [13, 14, 15], # Ring
            [17, 18, 19]  # Pinky
        ]
        
        angles = []
        for joints in finger_joints:
            try:
                angle = get_angle(
                    landmarks[joints[0]], 
                    landmarks[joints[1]], 
                    landmarks[joints[2]]
                )
                angles.append(angle)
            except:
                angles.append(0)
        
        return angles
    
    def detect_thumbs_up(self, landmarks, handedness):
        """
        Detect thumbs up gesture using multiple geometric features.
        Works from multiple angles by analyzing finger positions and orientations.
        """
        try:
            # Get key landmarks
            thumb_tip = landmarks[4]
            thumb_mcp = landmarks[2]
            index_tip = landmarks[8]
            index_mcp = landmarks[5]
            middle_tip = landmarks[12]
            wrist = landmarks[0]
            
            # Calculate relative positions
            thumb_height = abs(thumb_tip.y - wrist.y)
            index_height = abs(index_tip.y - wrist.y)
            middle_height = abs(middle_tip.y - wrist.y)
            
            # Check if thumb is extended upward relative to other fingers
            thumb_extended = thumb_height > max(index_height, middle_height) * 0.8
            
            # Check finger curl states using angles
            angles = self.calculate_finger_angles(landmarks)
            thumb_angle, index_angle, middle_angle, ring_angle, pinky_angle = angles
            
            # Fingers should be curled (higher angles indicate more curl)
            fingers_curled = (
                index_angle > 90 and 
                middle_angle > 90 and 
                ring_angle > 90 and 
                pinky_angle > 90
            )
            
            # Additional check: thumb should be pointing away from palm
            thumb_distance_from_palm = np.sqrt(
                (thumb_tip.x - index_mcp.x)**2 + 
                (thumb_tip.y - index_mcp.y)**2
            )
            
            # Combine all conditions
            is_thumbs_up = (
                thumb_extended and 
                fingers_curled and 
                thumb_distance_from_palm > 0.1
            )
            
            return is_thumbs_up, 0.9 if is_thumbs_up else 0.1
            
        except Exception as e:
            return False, 0.0
    
    def detect_peace_sign(self, landmarks, handedness):
        """
        Detect peace sign (V sign) gesture.
        Analyzes index and middle finger extension while other fingers are curled.
        """
        try:
            # Get key landmarks
            index_tip = landmarks[8]
            index_pip = landmarks[6]
            middle_tip = landmarks[12]
            middle_pip = landmarks[10]
            ring_tip = landmarks[16]
            pinky_tip = landmarks[20]
            wrist = landmarks[0]
            
            # Calculate finger extensions
            index_extended = abs(index_tip.y - wrist.y) > abs(index_pip.y - wrist.y)
            middle_extended = abs(middle_tip.y - wrist.y) > abs(middle_pip.y - wrist.y)
            
            # Check if index and middle fingers are spread apart
            finger_spread = np.sqrt(
                (index_tip.x - middle_tip.x)**2 + 
                (index_tip.y - middle_tip.y)**2
            )
            
            # Calculate finger angles
            angles = self.calculate_finger_angles(landmarks)
            thumb_angle, index_angle, middle_angle, ring_angle, pinky_angle = angles
            
            # Index and middle should be extended (lower angles)
            # Ring and pinky should be curled (higher angles)
            fingers_correct = (
                index_angle < 60 and 
                middle_angle < 60 and 
                ring_angle > 90 and 
                pinky_angle > 90
            )
            
            # Check vertical separation between extended fingers
            vertical_separation = abs(index_tip.y - middle_tip.y) < 0.05
            
            is_peace = (
                index_extended and 
                middle_extended and 
                fingers_correct and 
                finger_spread > 0.05 and
                vertical_separation
            )
            
            return is_peace, 0.9 if is_peace else 0.1
            
        except Exception as e:
            return False, 0.0
    
    def recognize_gesture(self, image):
        """
        Main gesture recognition function.
        Returns gesture type and confidence score.
        """
        hand_landmarks, handedness = self.get_hand_landmarks(image)
        
        if not hand_landmarks:
            return None, 0.0
        
        best_gesture = None
        best_confidence = 0.0
        
        # Check all detected hands
        for i, landmarks in enumerate(hand_landmarks):
            hand_type = handedness[i].classification[0].label if handedness else "Unknown"
            
            # Check for thumbs up
            thumbs_up, thumbs_confidence = self.detect_thumbs_up(landmarks.landmark, hand_type)
            if thumbs_up and thumbs_confidence > best_confidence:
                best_gesture = "thumbs_up"
                best_confidence = thumbs_confidence
            
            # Check for peace sign
            peace, peace_confidence = self.detect_peace_sign(landmarks.landmark, hand_type)
            if peace and peace_confidence > best_confidence:
                best_gesture = "peace"
                best_confidence = peace_confidence
        
        return best_gesture, best_confidence
    
    def update_gesture_state(self, image):
        """
        Update timer state based on recognized gestures.
        Includes smoothing to prevent false positives.
        """
        gesture, confidence = self.recognize_gesture(image)
        
        # Add to gesture history for smoothing
        if confidence > self.confidence_threshold:
            self.gesture_history.append(gesture)
        else:
            self.gesture_history.append(None)
        
        # Determine smoothed gesture
        if len(self.gesture_history) == self.smoothing_frames:
            gesture_counts = {}
            for g in self.gesture_history:
                if g:
                    gesture_counts[g] = gesture_counts.get(g, 0) + 1
            
            # Require majority consensus
            required_count = self.smoothing_frames // 2 + 1
            smoothed_gesture = None
            
            for g, count in gesture_counts.items():
                if count >= required_count:
                    smoothed_gesture = g
                    break
            
            # Update timer state based on smoothed gesture
            current_time = time.time()
            
            if smoothed_gesture != self.last_gesture:
                if smoothed_gesture == "thumbs_up" and not self.is_running:
                    self.resume_timer()
                elif smoothed_gesture == "peace" and self.is_running:
                    self.pause_timer()
                
                self.last_gesture = smoothed_gesture
                self.gesture_change_time = current_time
        
        return gesture, confidence
    
    def resume_timer(self):
        """Resume the timer."""
        if not self.is_running:
            self.is_running = True
            if self.pause_time:
                # Resume from where we paused
                pause_duration = time.time() - self.pause_time
                self.start_time += pause_duration
                self.pause_time = None
            else:
                # Start fresh
                self.start_time = time.time()
    
    def pause_timer(self):
        """Pause the timer."""
        if self.is_running:
            self.is_running = False
            self.pause_time = time.time()
            if self.start_time:
                self.elapsed_time = self.pause_time - self.start_time
    
    def get_elapsed_time(self):
        """Get current elapsed time."""
        if self.is_running and self.start_time:
            return time.time() - self.start_time
        elif self.pause_time and self.start_time:
            return self.pause_time - self.start_time
        return 0.0
    
    def reset_timer(self):
        """Reset the timer to initial state."""
        self.is_running = False
        self.start_time = None
        self.elapsed_time = 0.0
        self.pause_time = None
        self.gesture_history.clear()
        self.last_gesture = None

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    """
    Calculate angle between three points
    a, b, c are lists/arrays of [x, y] coordinates
    b is the vertex point
    """
    a = np.array(a)  # First point
    b = np.array(b)  # Vertex point
    c = np.array(c)  # End point
    
    # Calculate vectors
    ba = a - b
    bc = c - b
    
    # Calculate angle using dot product
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    
    # Convert to degrees
    return np.degrees(angle)

# Function to calculate distance between two points
def calculate_distance(point1, point2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

# Function to detect jumping jack stage
def detect_jumping_jack_stage(left_wrist, right_wrist, left_ankle, right_ankle, 
                            left_shoulder, right_shoulder, left_hip, right_hip):
    """
    Detect if person is in 'up' or 'down' stage of jumping jack
    Up stage: arms raised, legs spread apart
    Down stage: arms down, legs together
    """
    
    # Calculate arm spread (distance between wrists relative to shoulders)
    wrist_distance = calculate_distance(left_wrist, right_wrist)
    shoulder_distance = calculate_distance(left_shoulder, right_shoulder)
    arm_spread_ratio = wrist_distance / shoulder_distance if shoulder_distance > 0 else 0
    
    # Calculate leg spread (distance between ankles relative to hips)
    ankle_distance = calculate_distance(left_ankle, right_ankle)
    hip_distance = calculate_distance(left_hip, right_hip)
    leg_spread_ratio = ankle_distance / hip_distance if hip_distance > 0 else 0
    
    # Calculate arm height (average wrist height relative to shoulders)
    avg_wrist_y = (left_wrist[1] + right_wrist[1]) / 2
    avg_shoulder_y = (left_shoulder[1] + right_shoulder[1]) / 2
    arm_height_ratio = (avg_shoulder_y - avg_wrist_y)  # Positive when arms are raised
    
    # Thresholds for detection (adjust based on testing)
    ARM_SPREAD_UP_THRESHOLD = 1.8    # Arms spread wide
    ARM_SPREAD_DOWN_THRESHOLD = 1.2  # Arms close to body
    LEG_SPREAD_UP_THRESHOLD = 1.8    # Legs spread apart
    LEG_SPREAD_DOWN_THRESHOLD = 1.2  # Legs close together
    ARM_HEIGHT_THRESHOLD = 0.1       # Arms raised above shoulders
    
    # Determine stage based on arm and leg positions
    arms_up = (arm_spread_ratio > ARM_SPREAD_UP_THRESHOLD and arm_height_ratio > ARM_HEIGHT_THRESHOLD)
    arms_down = (arm_spread_ratio < ARM_SPREAD_DOWN_THRESHOLD and arm_height_ratio < -ARM_HEIGHT_THRESHOLD)
    
    legs_apart = leg_spread_ratio > LEG_SPREAD_UP_THRESHOLD
    legs_together = leg_spread_ratio < LEG_SPREAD_DOWN_THRESHOLD
    
    if arms_up and legs_apart:
        return 'up', arm_spread_ratio, leg_spread_ratio, arm_height_ratio
    elif arms_down and legs_together:
        return 'down', arm_spread_ratio, leg_spread_ratio, arm_height_ratio
    else:
        return 'transition', arm_spread_ratio, leg_spread_ratio, arm_height_ratio

def format_time(seconds):
    """Format seconds into MM:SS format."""
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes:02d}:{seconds:02d}"

# Main application
def main():
    # Initialize video capture
    cap = cv2.VideoCapture(0)
    
    # Initialize gesture timer
    gesture_timer = GestureTimer(confidence_threshold=0.7, smoothing_frames=3)
    
    # Jumping jack counter variables
    jj_counter = 0
    jj_stage = None
    
    # Set up Mediapipe pose instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, frame = cap.read()
            
            if not ret:
                print("Failed to grab frame")
                break
                
            # Get frame dimensions
            frame_height, frame_width = frame.shape[:2]
            
            # Update gesture timer state
            current_gesture, gesture_confidence = gesture_timer.update_gesture_state(frame)
            
            # Recolor image to RGB for pose detection
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            
            # Make pose detection
            results = pose.process(image)
            
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Initialize variables
            left_knee_angle = 0
            right_knee_angle = 0
            current_stage = 'none'
            arm_spread_ratio = 0
            leg_spread_ratio = 0
            arm_height_ratio = 0
            
            # Extract landmarks
            try:
                if results.pose_landmarks:
                    landmarks = results.pose_landmarks.landmark
                    
                    # Get coordinates for required landmarks (normalized)
                    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                    
                    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                 landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                    
                    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                 landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                    
                    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                               landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                    
                    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                                 landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
                    
                    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                 landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                                  landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]
                    
                    # Calculate knee angles for both legs (hip-knee-ankle)
                    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
                    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
                    
                    # Detect jumping jack stage
                    current_stage, arm_spread_ratio, leg_spread_ratio, arm_height_ratio = detect_jumping_jack_stage(
                        left_wrist, right_wrist, left_ankle, right_ankle,
                        left_shoulder, right_shoulder, left_hip, right_hip
                    )
                    
                    # Jumping jack counting logic - ONLY COUNT WHEN TIMER IS RUNNING
                    if gesture_timer.is_running:
                        if current_stage == 'up':
                            jj_stage = 'up'
                        elif current_stage == 'down' and jj_stage == 'up':
                            jj_stage = 'down'
                            jj_counter += 1
                            print(f"Jumping jack completed! Count: {jj_counter}")
                    
                    # Convert coordinates to pixel coordinates for visualization
                    left_wrist_coords = tuple(np.multiply(left_wrist, [frame_width, frame_height]).astype(int))
                    right_wrist_coords = tuple(np.multiply(right_wrist, [frame_width, frame_height]).astype(int))
                    left_ankle_coords = tuple(np.multiply(left_ankle, [frame_width, frame_height]).astype(int))
                    right_ankle_coords = tuple(np.multiply(right_ankle, [frame_width, frame_height]).astype(int))
                    left_knee_coords = tuple(np.multiply(left_knee, [frame_width, frame_height]).astype(int))
                    right_knee_coords = tuple(np.multiply(right_knee, [frame_width, frame_height]).astype(int))
                    
                    # Draw indicators based on current stage
                    if current_stage == 'up':
                        # Highlight key points in green when in up position
                        cv2.circle(image, left_wrist_coords, 8, (0, 255, 0), -1)
                        cv2.circle(image, right_wrist_coords, 8, (0, 255, 0), -1)
                        cv2.circle(image, left_ankle_coords, 8, (0, 255, 0), -1)
                        cv2.circle(image, right_ankle_coords, 8, (0, 255, 0), -1)
                    elif current_stage == 'down':
                        # Highlight key points in blue when in down position
                        cv2.circle(image, left_wrist_coords, 8, (255, 0, 0), -1)
                        cv2.circle(image, right_wrist_coords, 8, (255, 0, 0), -1)
                        cv2.circle(image, left_ankle_coords, 8, (255, 0, 0), -1)
                        cv2.circle(image, right_ankle_coords, 8, (255, 0, 0), -1)
                    else:
                        # Neutral color during transition
                        cv2.circle(image, left_wrist_coords, 6, (0, 255, 255), -1)
                        cv2.circle(image, right_wrist_coords, 6, (0, 255, 255), -1)
                        cv2.circle(image, left_ankle_coords, 6, (0, 255, 255), -1)
                        cv2.circle(image, right_ankle_coords, 6, (0, 255, 255), -1)
                    
                    # Draw knee angle indicators
                    cv2.circle(image, left_knee_coords, 6, (255, 255, 0), -1)
                    cv2.circle(image, right_knee_coords, 6, (255, 255, 0), -1)
                    
                    # Display knee angles
                    cv2.putText(image, f'{int(left_knee_angle)}°',
                               (left_knee_coords[0] - 30, left_knee_coords[1] - 15),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    
                    cv2.putText(image, f'{int(right_knee_angle)}°',
                               (right_knee_coords[0] + 10, right_knee_coords[1] - 15),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
                    
            except Exception as e:
                print("Error:", e)
            
            # Setup main status display (expanded for timer)
            cv2.rectangle(image, (10, 10), (600, 280), (50, 50, 50), -1)
            cv2.rectangle(image, (10, 10), (600, 280), (255, 255, 255), 2)
            
            # Display jumping jack count
            cv2.putText(image, 'JUMPING JACKS', (20, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(jj_counter), (20, 80), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3, cv2.LINE_AA)
            
            # Display current stage
            stage_color = (0, 255, 0) if current_stage == 'up' else (255, 0, 0) if current_stage == 'down' else (0, 255, 255)
            cv2.putText(image, 'STAGE', (200, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, current_stage.upper(), (200, 75), 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, stage_color, 2, cv2.LINE_AA)
            
            # Display timer information
            timer_color = (0, 255, 0) if gesture_timer.is_running else (255, 255, 0)
            timer_status = "RUNNING" if gesture_timer.is_running else "PAUSED"
            elapsed_time = gesture_timer.get_elapsed_time()
            
            cv2.putText(image, 'TIMER', (350, 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, timer_status, (350, 65), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, timer_color, 2, cv2.LINE_AA)
            cv2.putText(image, format_time(elapsed_time), (350, 90), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Display gesture information
            cv2.putText(image, 'GESTURE CONTROL', (20, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
            
            gesture_text = "None"
            gesture_color = (128, 128, 128)
            
            if current_gesture == "thumbs_up":
                gesture_text = "👍 THUMBS UP"
                gesture_color = (0, 255, 0)
            elif current_gesture == "peace":
                gesture_text = "✌️ PEACE SIGN"
                gesture_color = (255, 255, 0)
            
            cv2.putText(image, gesture_text, (20, 145), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, gesture_color, 2, cv2.LINE_AA)
            cv2.putText(image, f'Confidence: {gesture_confidence:.2f}', (20, 170), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Display knee angles
            cv2.putText(image, 'KNEE ANGLES', (350, 120), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'L: {int(left_knee_angle)}°', (350, 145), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(image, f'R: {int(right_knee_angle)}°', (350, 165), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2, cv2.LINE_AA)
            
            # Display detection metrics
            cv2.putText(image, 'DETECTION METRICS', (20, 200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, f'Arm Spread: {arm_spread_ratio:.2f}', (20, 220), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, f'Leg Spread: {leg_spread_ratio:.2f}', (20, 240), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, f'Arm Height: {arm_height_ratio:.2f}', (20, 260), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            
            # Display current internal stage
            internal_stage = jj_stage if jj_stage else 'None'
            cv2.putText(image, f'Internal: {internal_stage}', (350, 200), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 200), 1, cv2.LINE_AA)
            
            # Render pose detections
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
            
            # Draw hand landmarks for gesture debugging (optional)
            hand_landmarks, handedness = gesture_timer.get_hand_landmarks(image)
            if hand_landmarks:
                for hand_landmark in hand_landmarks:
                    mp_drawing.draw_landmarks(
                        image, hand_landmark, mp_hands.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),
                        mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2)
                    )
            
            # Add instructions
            instruction_y = frame_height - 60
            cv2.putText(image, 'CONTROLS: 👍 Thumbs Up = Resume Timer | ✌️ Peace Sign = Pause Timer | Q = Quit', 
                        (10, instruction_y), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1, cv2.LINE_AA)
            cv2.putText(image, 'NOTE: Reps only count when timer is running!', 
                        (10, instruction_y + 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1, cv2.LINE_AA)
            cv2.putText(image, 'Hold gesture steady for 2-3 seconds for best recognition', 
                        (10, instruction_y + 40), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (200, 200, 200), 1, cv2.LINE_AA)
            
            # Display the image
            cv2.imshow('Gesture-Controlled Jumping Jack Counter', image)
            
            # Handle keyboard input
            key = cv2.waitKey(10) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('r'):  # Reset timer manually
                gesture_timer.reset_timer()
                jj_counter = 0
                jj_stage = None
                print("Timer and counter reset!")
    
    # Print final statistics
    print(f"\nFinal Jumping Jack Results:")
    print(f"Total Jumping Jacks: {jj_counter}")
    print(f"Total Workout Time: {format_time(gesture_timer.get_elapsed_time())}")
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

KeyboardInterrupt: 